In [1]:
import numpy as np
import tensorflow as tf
import torch
import torch.nn as nn
import torch.nn.functional as F
import sys

import pyarrow as pa
import pyarrow.parquet as pq
import pandas as pd
from pandas import DataFrame
from pandas.core.dtypes import common as com
from core.loader import Loader
from models.model_wrapper import ModelWrapper

from tensorflow.python.client import device_lib

for device in device_lib.list_local_devices():
    print(device.physical_device_desc)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# TODO: Zkusit 3 stupně na NN, SVM a porovnat s LightGBM + XGBoost

# 4 x Malware
# 4x  Phishing
# + DGA (jeden stupen)

2025-04-29 19:38:21.895803: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-29 19:38:21.895836: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-29 19:38:21.896908: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-29 19:38:21.903361: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-29 19:38:22.623829: W tensorflow/compiler/tf2


device: 0, name: NVIDIA GeForce RTX 3050 Ti Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


2025-04-29 19:38:25.147111: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-04-29 19:38:25.188148: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-04-29 19:38:25.191140: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [ ]:

malicious_label = "malware"

# #############################################################
# EDIT this to specify benign / malicious datasets to use     #
# #############################################################
benign_dataset_filenames = [
    f'parkets/testdata/2405_clftest_benign_filtered.parquet'
        
]
malicious_dataset_filenames = [
    f'parkets/testdata/2405_clftest_{malicious_label}_filtered.parquet'
]

benign_dataset_filenames_train = [
    'parkets/benign/benign_2312_anonymized_HTML.parquet', 
    'parkets/benign/umbrella_benign_FINISHED_HTML.parquet'
        
]
malicious_dataset_filenames_train = [
    f'parkets/{malicious_label}_2406_strict_HTML.parquet'
]


benign_label = "benign"
# lex-dga-830k-pick.parquet LABEL je jmeno rodiny
#  'parkets/malware_2406_strict_HTML.parquet'


# #############################################################
# EDIT this for to set appropriate labels (malware, dga, ...) #
# #############################################################

# #############################################################

class_map = {benign_label: 0, malicious_label: 1}
# print labels from malicious datasets

loader = Loader(benign_dataset_filenames, malicious_dataset_filenames, benign_label=benign_label, malicious_label=malicious_label, subsample=1.0)

train_loader = Loader(benign_dataset_filenames_train, malicious_dataset_filenames_train, benign_label=benign_label, malicious_label=malicious_label, subsample=0.5)


Length of df:  13612
Columns:  Index(['label', 'dns_has_dnskey', 'dns_A_count', 'dns_AAAA_count',
       'dns_MX_count', 'dns_NS_count', 'dns_TXT_count', 'dns_SOA_count',
       'dns_CNAME_count', 'dns_zone_level',
       ...
       'html_index_of', 'html_document_write', 'html_get', 'html_find',
       'html_document_create_element', 'html_window_set_timeout',
       'html_window_set_interval', 'html_hex_encoding',
       'html_unicode_encoding', 'html_long_variable_name'],
      dtype='object', length=264)


In [ ]:
train_df = train_loader.load()
df = loader.load()




df = pd.concat([df.sample(frac=1.0), train_df.sample(frac=0.05)], ignore_index=True)

# print df length
print("Length of df: ", len(df))
print("Columns: ", df.columns)

Length of df:  8307
Columns:  Index(['label', 'dns_has_dnskey', 'dns_A_count', 'dns_AAAA_count',
       'dns_MX_count', 'dns_NS_count', 'dns_TXT_count', 'dns_SOA_count',
       'dns_CNAME_count', 'dns_zone_level',
       ...
       'html_index_of', 'html_document_write', 'html_get', 'html_find',
       'html_document_create_element', 'html_window_set_timeout',
       'html_window_set_interval', 'html_hex_encoding',
       'html_unicode_encoding', 'html_long_variable_name'],
      dtype='object', length=264)


In [16]:
from core.segmenter import Segmenter

# Define the aggregates that needs to be created

aggregates = [
    ["lex_"],
    ["lex_", "dns_", "ip_", "geo_"],
    ["lex_", "dns_", "ip_", "tls_", "geo_", "rdap_"],
]

segmenter = Segmenter(df)
segmenter.create_base_subsets() # create base subsets
segmenter.create_aggregated_subsets(aggregates)
subset_dfs = segmenter.get_aggregated_subsets()

/home/poli/Desktop/git/deep_domain_detection/src/core/segmenter.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df["label"] = self.df["label"].copy()
/home/poli/Desktop/git/deep_domain_detection/src/core/segmenter.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df["label"] = self.df["label"].copy()
/home/poli/Desktop/git/deep_domain_detection/src/core/segmenter.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

In [ ]:
from sklearn.model_selection import train_test_split

i = 1
for prefix, subset_df in subset_dfs.items():
    X = subset_df.drop('label', axis=1)
    y = subset_df['label'].map(class_map)
        
    dump = {
        "X_test": X.to_numpy(),
        "Y_test": y.to_numpy(),
    }

    # save using pickle to sata
    import pickle

    with open(f"./data/verification_stage_{i}_{malicious_label}.pkl", "wb") as f:
        pickle.dump(dump, f)
        
    print(f'Saved verification stage {i} to ./data/verification_stage_{i}_{malicious_label}.pkl')  
    i += 1


Saved verification stage 1 to ./data/verification_stage_1_phishing.pkl
Saved verification stage 2 to ./data/verification_stage_2_phishing.pkl
Saved verification stage 3 to ./data/verification_stage_3_phishing.pkl
